# 🚀 Projeto de Portfólio: Otimização de Atendimento ao Cliente com IA Generativa

*   **Autor:** Natasha Brandão
*   **Data:** 07/08/2025

---

### 1. Contexto do Problema

No cenário digital atual, empresas de grande porte lidam com um volume massivo de interações de clientes em redes sociais. Analisar e responder a cada tweet de forma eficiente é um desafio logístico e de custo. Respostas lentas ou inadequadas podem levar à insatisfação do cliente, enquanto a análise manual de milhares de mensagens é impraticável.

### 2. Objetivo do Projeto

Este projeto tem como objetivo desenvolver um protótipo de ponta a ponta de um sistema de Inteligência Artificial para otimizar o atendimento ao cliente. A solução irá:

1.  **Analisar e classificar** automaticamente os tweets dos clientes em categorias (reclamação, elogio, dúvida).
2.  **Detectar o sentimento** (positivo, negativo, neutro) de cada mensagem.
3.  **Gerar respostas contextuais e adequadas** usando um modelo de linguagem avançado (LLM).
4.  **(Etapa Futura) Apresentar os resultados** de forma interativa através de um dashboard.

### ✅ Habilidades e Tecnologias Demonstradas

Este projeto serve como uma demonstração prática de um fluxo de trabalho de ponta a ponta em Ciência de Dados e IA, cobrindo as seguintes competências técnicas e analíticas:

*   **Engenharia de Features e Análise de Dados:** Análise exploratória (EDA), limpeza de dados textuais, e criação de features relevantes (e.g., tipo de mensagem, sentimento) a partir de dados não estruturados.
*   **Processamento de Linguagem Natural (NLP):** Técnicas de pré-processamento, análise de sentimento com NLTK (VADER) e modelagem de tópicos implícita através da geração de respostas.
*   **IA Generativa e LLMs:** Utilização da biblioteca `transformers` (Hugging Face) para implementar e controlar um Large Language Model (como o GPT-2) na geração de texto contextual.
*   **Modelagem de Machine Learning:** Aplicação de modelos pré-treinados para tarefas de NLP e prototipagem de uma solução de IA.
*   **Visualização de Dados e BI:** Criação de visualizações para extrair insights e (na etapa final) desenvolvimento de um dashboard interativo para apresentar os resultados de forma clara.
*   **Pensamento Estratégico:** Tradução de um problema de negócio (otimização de atendimento) em um plano de projeto técnico, executado de forma sequencial e lógica.

### Dataset
*   **Fonte:** [Customer Support on Twitter](https://www.kaggle.com/datasets/thoughtvector/customer-support-on-twitter)
*   **Descrição:** Um grande dataset do Kaggle contendo quase 3 milhões de tweets de e para grandes marcas.

In [ ]:
# --- Configuração do Ambiente e Bibliotecas Essenciais ---

# Análise de Dados e Manipulação
import pandas as pd
import numpy as np

# Visualização de Dados
import matplotlib.pyplot as plt
import seaborn as sns

# Utilitários do Sistema
import os

# Configurações de Visualização (opcional, para deixar os gráficos mais bonitos)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# --- Verificação Inicial dos Arquivos ---
# Este loop inicial nos ajuda a confirmar os caminhos dos arquivos de dados disponíveis.
print("Arquivos de dados disponíveis:")
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Parte 1: Análise Exploratória e Engenharia de Features

## 🧠 Objetivo da Análise Exploratória de Dados (EDA)

Antes de construir qualquer modelo, precisamos entender profundamente nossos dados. O objetivo desta primeira fase é responder às seguintes perguntas-chave:

*   Qual é o volume total de interações no dataset?
*   Como as conversas entre clientes e empresas estão estruturadas?
*   Quais são os temas e palavras-chave mais recorrentes nas mensagens?
*   É possível identificar automaticamente se um tweet é uma pergunta, reclamação ou elogio?
*   Como o volume de atendimentos se distribui ao longo do tempo? Existe alguma sazonalidade?

---
### 🛠️ Etapas do EDA

#### 1. Carregando os Dados

In [ ]:
# Importando pandas para manipulação de dados
import pandas as pd

# Carregando o conjunto de dados principal em um DataFrame do pandas.
# O arquivo está no formato CSV (Comma-Separated Values).
df = pd.read_csv('/kaggle/input/customer-support-on-twitter/twcs/twcs.csv')

# Exibindo as primeiras 5 linhas para uma inspeção inicial da estrutura e do conteúdo.
# Isso nos ajuda a ter uma primeira impressão dos dados com os quais estamos trabalhando.
df.head()

### 2. Análise Estrutural e Validação dos Dados

Nesta etapa, realizamos uma verificação fundamental para entender as dimensões, os tipos de dados e a integridade do nosso DataFrame. Isso é crucial para planejar as etapas de limpeza e pré-processamento.

Vamos verificar:
*   **Dimensões:** Número de linhas e colunas.
*   **Tipos de Dados (Dtypes):** Se as colunas foram lidas corretamente (números, texto, data, etc.).
*   **Valores Nulos:** A quantidade de dados ausentes em cada coluna.

In [ ]:
# --- Verificação Técnica do DataFrame ---

# Um resumo completo da estrutura, incluindo tipos de dados e uso de memória.
print("--- Informações Gerais do DataFrame (df.info()) ---")
df.info()

print("\n" + "="*50 + "\n")

# Contagem de valores nulos para cada coluna.
print("--- Contagem de Valores Nulos por Coluna ---")
print(df.isnull().sum())

### Principais Observações:

1.  **Volume Massivo:** O dataset contém aproximadamente **2.8 milhões de tweets**, confirmando a necessidade de uma solução automatizada.
2.  **Datas como Texto:** A coluna `created_at` foi lida como `object` (texto). Precisaremos convertê-la para um formato de data (`datetime`) para realizar análises temporais.
3.  **Valores Nulos Estratégicos:** As colunas `response_tweet_id` e `in_response_to_tweet_id` possuem um grande número de valores nulos. Isso é uma informação valiosa: um tweet com `in_response_to_tweet_id` nulo provavelmente é o **início de uma conversa**, ou seja, o primeiro contato de um cliente.

### 3. Reconstruindo o Fio da Conversa

O DataFrame apresenta os tweets de forma "plana", mas eles fazem parte de conversas encadeadas. Para entender a dinâmica do atendimento, é essencial reconstruir um exemplo de interação `Cliente -> Empresa`. O código abaixo faz exatamente isso.

In [ ]:
# --- Lógica para Exibir uma Conversa de Exemplo ---

# 1. Identificar os autores que são empresas (aqueles que enviam tweets "outbound").
# A coluna 'inbound' nos diz se o tweet foi direcionado à empresa (True) ou enviado pela empresa (False).
empresas = df[df['inbound'] == False]['author_id'].unique()

# 2. Selecionar uma amostra aleatória de um tweet que seja de um cliente (NÃO é de uma empresa) e que tenha recebido uma resposta.
tweet_do_cliente = df[(~df['author_id'].isin(empresas)) & (df['response_tweet_id'].notna())].sample(1)

# 3. Extrair os IDs do tweet do cliente e da resposta da empresa.
id_tweet_cliente = tweet_do_cliente['tweet_id'].values[0]
id_resposta_empresa = tweet_do_cliente['response_tweet_id'].values[0]

# 4. Localizar o texto da resposta da empresa usando o ID obtido.
# É necessário converter o ID da resposta para inteiro (int) para corresponder ao tipo de 'tweet_id'.
texto_cliente = tweet_do_cliente['text'].values[0]
resposta_empresa = df[df['tweet_id'] == int(id_resposta_empresa)]

# 5. Exibir a conversa reconstruída.
print(f"TWEET DO CLIENTE: {texto_cliente}")

if not resposta_empresa.empty:
    print(f"RESPOSTA DA EMPRESA: {resposta_empresa['text'].values[0]}")
else:
    print("RESPOSTA DA EMPRESA: Não encontrada.")


# Parte 2: Pré-processamento e Limpeza dos Dados

Com os insights da EDA, agora preparamos os dados para a modelagem. Isso envolve duas etapas principais: filtrar o dataset para o período de maior relevância e limpar o texto dos tweets para remover ruídos.

### 1. Filtragem Temporal e Limpeza de Texto

In [ ]:
# --- ETAPA DE PREPARAÇÃO ---

# Importando as bibliotecas necessárias para esta seção.
import pandas as pd
import re  # Biblioteca para expressões regulares, essencial para a limpeza de texto.
from nltk.corpus import stopwords
import nltk

# 1. Converter a coluna 'created_at' para o formato de data.
#    Especificamos o 'format' para melhorar a performance e remover avisos.
#    O formato corresponde exatamente à estrutura da data no CSV (Ex: 'Tue Oct 31 22:10:47 +0000 2017')
df['created_at'] = pd.to_datetime(df['created_at'], format='%a %b %d %H:%M:%S %z %Y', errors='coerce')

# 2. Filtrar o DataFrame para manter apenas os dados a partir de 2017, período de maior atividade.
df_filtered = df[df['created_at'].dt.year >= 2017].copy()

# 3. Baixar a lista de stopwords (se ainda não tiver sido baixada nesta sessão)
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# 4. Definir a função para limpar o texto dos tweets.
#    Esta função irá remover elementos que não agregam valor semântico, como URLs, menções e stopwords.
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text) # Remove URLs
    text = re.sub(r'@\w+', '', text) # Remove menções (@usuario)
    text = re.sub(r'#', '', text) # Remove o símbolo de hashtag
    text = re.sub(r'[^a-z\s]', '', text) # Remove pontuações e números
    text = ' '.join([word for word in text.split() if word not in stop_words]) # Remove 'stopwords'
    return text

# 5. Aplicar a função de limpeza na coluna de texto para criar a coluna 'clean_text'.
df_filtered['clean_text'] = df_filtered['text'].apply(clean_text)

# 6. Exibir uma amostra do resultado para validar a limpeza.
print("\n--- Resultado da Limpeza ---")
print(df_filtered[['text', 'clean_text']].head())

### 2. Análise do Comprimento das Mensagens

Uma análise rápida do comprimento dos tweets nos ajuda a entender a natureza das interações. São mensagens curtas e diretas ou longas e detalhadas? Isso pode nos dar pistas sobre a complexidade dos problemas dos clientes.

In [ ]:
# --- Análise do Comprimento dos Tweets ---

# 1. Criar uma nova coluna 'comprimento_texto' no DataFrame original 'df'.
#    Usamos o DataFrame original para ter uma visão completa, antes de qualquer filtragem.
df['comprimento_texto'] = df['text'].str.len()

# 2. Criar um histograma para visualizar a distribuição dos comprimentos.
#    O histograma agrupa os tweets por faixas de comprimento e mostra a frequência de cada faixa.
plt.figure(figsize=(14, 7))
sns.histplot(df['comprimento_texto'], bins=50, kde=True, color='skyblue')
plt.title('Distribuição do Comprimento dos Tweets (em caracteres)', fontsize=16)
plt.xlabel('Comprimento do Texto (Caracteres)', fontsize=12)
plt.ylabel('Frequência (Quantidade de Tweets)', fontsize=12)
plt.grid(True)
plt.show()

#### Insights:

*   **Pico em ~140 Caracteres:** A grande maioria dos tweets se concentra em torno de 100 a 140 caracteres. Isso é consistente com o limite histórico de 140 caracteres do Twitter, indicando que o dataset foi coletado majoritariamente nesse período.
*   **Cauda Longa à Direita:** Existe uma pequena quantidade de tweets mais longos, provavelmente coletados após a mudança do Twitter para 280 caracteres.

Esta análise confirma que estamos lidando com textos curtos e diretos, típicos de redes sociais, o que guiará a escolha das nossas técnicas de NLP.

### 3. Análise da Distribuição por Ano

Seguindo a hipótese de que os dados podem não ter sido coletados de forma contínua, vamos primeiro investigar o volume de tweets em cada ano. Esta análise macro nos dará uma visão clara da relevância de cada período e se existem lacunas na coleta de dados.

In [ ]:
# --- Análise da Contagem de Tweets por Ano ---

# 1. Garantir que a coluna 'created_at' está no formato de data.
df['created_at'] = pd.to_datetime(df['created_at'], format='%a %b %d %H:%M:%S %z %Y', errors='coerce')

# 2. Extrair o ano de cada tweet para uma nova coluna.
df['year'] = df['created_at'].dt.year

# 3. Contar o número de tweets para cada ano e ordenar os resultados.
tweets_por_ano = df['year'].value_counts().sort_index()

# 4. Exibir os resultados em um formato claro.
print("--- Contagem de Tweets por Ano ---")

# Formata a coluna para ser um número inteiro com separador de vírgula
tabela_formatada = tweets_por_ano.to_frame(name='Quantidade de Tweets')
tabela_formatada['Quantidade de Tweets'] = tabela_formatada['Quantidade de Tweets'].map('{:,.0f}'.format)

print(tabela_formatada.to_markdown())

# 5. Criar um gráfico de linha.
plt.figure(figsize=(14, 8))
plt.plot(tweets_por_ano.index, tweets_por_ano.values, marker='o', linestyle='-', color='teal')
plt.title('Crescimento Exponencial de Tweets no Dataset (Escala Logarítmica)', fontsize=16)
plt.xlabel('Ano', fontsize=12)
plt.ylabel('Quantidade de Tweets (Escala Log)', fontsize=12)

# 6. Mudar a escala do eixo Y para logarítmica para visualizar melhor os valores baixos.
plt.yscale('log')

plt.grid(True, which="both", ls="--")

# 7. Adicionar os rótulos de dados (data labels) em cada ponto do gráfico.
#    Este loop percorre cada ano e sua respectiva contagem de tweets.
for ano, quantidade in tweets_por_ano.items():
    # O comando 'plt.text' posiciona um texto nas coordenadas (x, y)
    # Adicionamos um pequeno deslocamento vertical para o texto não sobrepor o ponto.
    plt.text(ano, quantidade * 1.1, f'{quantidade:,.0f}', ha='center', va='bottom', fontsize=9)
    # '{:,.0f}' formata o número com vírgulas e sem casas decimais.

plt.show()

### 4. Análise Detalhada da Série Temporal (Diária)

A análise anual confirmou de forma conclusiva que **a atividade do dataset se concentra quase que inteiramente em 2017**. Agora, vamos aplicar um "zoom" nesse período, plotando o volume de tweets em uma base diária. Isso nos permitirá observar flutuações e padrões de curto prazo com mais clareza.

In [ ]:
# --- Análise da Série Temporal de Tweets (Diária) ---

# 1. Agrupar a contagem de tweets por dia.
#    O método 'resample('D')' é uma forma poderosa do pandas para reagrupar dados de séries temporais.
tweets_por_dia = df.set_index('created_at')['tweet_id'].resample('D').count()

# 2. Gerar um gráfico de linha para visualizar a tendência diária.
plt.figure(figsize=(16, 7))
tweets_por_dia.plot(color='c')
plt.title('Volume de Tweets por Dia ao Longo do Tempo', fontsize=16)
plt.xlabel('Data', fontsize=12)
plt.ylabel('Quantidade de Tweets', fontsize=12)
plt.grid(True)
plt.xlim(pd.Timestamp('2017-01-01'), pd.Timestamp('2017-12-31')) # Adicionando um zoom no gráfico
plt.show()

### Insights da Análise Temporal

As duas análises temporais, a anual e a diária, nos fornecem uma visão completa e poderosa sobre a distribuição dos dados, revelando dois insights críticos:

1.  **Concentração Anual (Gráfico Logarítmico):** A primeira análise deixa claro que o dataset não é uma coleta contínua. Pelo contrário, a atividade é **massivamente concentrada no ano de 2017**. O uso da escala logarítmica nos permite ver o crescimento nos anos anteriores, mas confirma que esses volumes são insignificantes em comparação com o salto exponencial no último ano.

2.  **Intensificação da Coleta em 2017 (Gráfico Diário):** Ao dar um "zoom" em 2017, descobrimos um padrão ainda mais específico: a coleta de dados não foi uniforme ao longo do ano. Houve uma **explosão drástica no volume de tweets a partir de Outubro**, indicando que a grande maioria dos dados relevantes se concentra no **último trimestre (Q4) de 2017**.

**Conclusão e Justificativa da Ação:**
Estas descobertas validam de forma irrefutável a nossa decisão, tomada na etapa de pré-processamento, de **filtrar o dataset para focar em 2017**. Esta ação foi essencial para garantir que nossa análise e futura modelagem sejam feitas sobre o conjunto de dados mais denso, consistente e relevante, otimizando o processamento e a qualidade dos resultados.

Com a exploração temporal concluída, podemos prosseguir com confiança para a próxima fase: a criação de features a partir do nosso dataset limpo e focado.

# Parte 3: Engenharia de Features - Classificação de Intenção

Com um dataset limpo e filtrado, iniciamos a Engenharia de Features. Nosso objetivo é transformar o texto não estruturado dos tweets em uma **feature categórica e estruturada** que descreva a **intenção** do cliente. Isso é fundamental para que nossos modelos possam entender o contexto de cada mensagem.

### 1. Classificação de Intenção por Regras

Como primeira abordagem, implementaremos um classificador simples baseado em regras e palavras-chave. Este método tem grandes vantagens para a prototipagem:

*   **Rapidez:** É computacionalmente muito eficiente.
*   **Interpretabilidade:** As regras são claras e fáceis de entender e ajustar.
*   **Baseline:** Cria uma base de referência sólida para, futuramente, comparar com modelos de Machine Learning mais complexos.

As intenções que buscaremos identificar são: `RECLAMAÇÃO`, `DÚVIDA`, `ELOGIO` e `OUTRO`.

In [ ]:
# --- Função de Classificação e Aplicação ---

# 1. Definição da função de classificação baseada em palavras-chave.
def classificar_tipo(texto):
    """Classifica um texto em 'reclamacao', 'duvida', 'elogio' ou 'outro'."""
    texto = str(texto).lower()
    
    # Listas de palavras-gatilho para cada categoria.
    palavras_reclamacao = ['not working', 'problem', 'issue', 'broken', "doesn't work", 'worst', 'unacceptable', 'fail']
    palavras_elogio = ['thanks', 'great', 'awesome', 'love', 'excellent', 'perfect', 'thank you', 'kudos']
    
    # A ordem da verificação é importante para a lógica.
    if any(p in texto for p in palavras_reclamacao):
        return 'reclamacao'
    
    # Assumimos que a presença de '?' indica uma dúvida.
    elif '?' in texto:
        return 'dúvida'
        
    elif any(p in texto for p in palavras_elogio):
        return 'elogio'
    
    # Se nenhuma das condições anteriores for atendida, classificamos como 'outro'.
    else:
        return 'outro'

# 2. Aplicar a função na coluna de texto do nosso DataFrame FILTRADO e LIMPO.
#    <<< CORREÇÃO CRÍTICA: Usamos 'df_filtered' para manter a consistência do projeto.
df_filtered['tipo'] = df_filtered['text'].apply(classificar_tipo)

# 3. Exibir a distribuição das novas categorias.
print("--- Distribuição das Intenções dos Tweets ---")
print(df_filtered['tipo'].value_counts())

In [ ]:
# --- Visualização da Distribuição das Intenções ---

# 1. Calcular a contagem de cada tipo.
tipo_counts = df_filtered['tipo'].value_counts()

# 2. Criar o gráfico de barras com Seaborn para um visual melhorado.
plt.figure(figsize=(12, 7))
ax = sns.barplot(x=tipo_counts.index, y=tipo_counts.values, palette='mako')
plt.title('Distribuição dos Tipos de Mensagem (Intenção)', fontsize=16)
plt.xlabel('Tipo de Intenção', fontsize=12)
plt.ylabel('Quantidade de Tweets', fontsize=12)

# 3. Adicionar rótulos de dados (data labels) em cima de cada barra.
#    Isso torna o gráfico muito mais fácil de ler.
for p in ax.patches:
    ax.annotate(f'{int(p.get_height()):,}', # Formata o número com vírgulas
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', 
                xytext=(0, 9), 
                textcoords='offset points')

plt.show()


#### Insights da Distribuição:

O gráfico acima nos fornece a primeira visão quantitativa sobre a natureza das interações no nosso dataset. Vemos uma distribuição clara:

1.  **Grande Volume de "Outro":** A categoria `outro` é a maior, com mais de 1.6 milhão de tweets. Isso é esperado, pois nosso classificador por regras é conservador e só categoriza tweets que contêm palavras-chave muito específicas. Este grupo representa uma vasta gama de interações que não são claramente reclamações, dúvidas ou elogios.

2.  **Dúvidas são Frequentes:** Com mais de 638 mil ocorrências, as `dúvidas` (identificadas pela presença de "?") são a segunda categoria mais comum. Isso indica que uma grande parte do suporte ao cliente em redes sociais envolve esclarecer questões dos usuários.

3.  **Elogios Superam Reclamações:** Um insight interessante é que o número de `elogios` (317 mil) é maior que o de `reclamações` (242 mil). Isso sugere que os clientes também usam o Twitter como um canal para expressar satisfação, e não apenas para relatar problemas.

**Implicação Estratégica:**
A distribuição nos mostra que, embora as reclamações sejam um ponto crítico, um sistema de IA eficaz também deve ser capaz de lidar com um grande volume de dúvidas e interações neutras, além de saber como responder a um elogio para reforçar a lealdade do cliente.

Com essa primeira classificação feita, o próximo passo é validá-la qualitativamente, observando amostras reais de cada categoria.```

### Por que esta análise é valiosa:

*   **Interpreta cada Categoria:** Ela não apenas olha para o gráfico como um todo, mas tira uma conclusão para cada uma das barras.
*   **Justifica os Resultados:** A análise explica *por que* é normal que a categoria `outro` seja a maior, demonstrando um entendimento do funcionamento do seu próprio classificador.
*   **Extrai um Insight de Negócio:** A observação de que "Elogios Superam Reclamações" é um insight não-trivial que pode ser valioso para uma equipe de marketing ou de produto.
*   **Cria uma Transição Lógica:** A última frase já prepara o terreno para a próxima etapa que você tem no seu notebook (a validação por amostragem), criando um fluxo narrativo perfeito.

### 2. Validação da Classificação

Um passo crucial em qualquer tarefa de classificação é verificar a qualidade dos resultados. Vamos extrair uma amostra aleatória de tweets classificados como `reclamacao` e `elogio` para avaliar se nosso classificador simples está se comportando como esperado.

In [ ]:
# --- Validação por Amostragem ---

print("--- Amostra de Tweets classificados como RECLAMAÇÃO ---")
# Usamos '.tolist()' para obter uma lista de strings, facilitando a exibição.
amostra_reclamacoes = df_filtered[df_filtered['tipo'] == 'reclamacao']['text'].sample(5).tolist()

# O loop 'enumerate' nos dá um contador 'i' (começando em 1) e o tweet.
for i, tweet in enumerate(amostra_reclamacoes, 1):
    print(f"{i}: {tweet}\n")


print("\n" + "="*50 + "\n")


print("--- Amostra de Tweets classificados como ELOGIO ---")
amostra_elogios = df_filtered[df_filtered['tipo'] == 'elogio']['text'].sample(5).tolist()
for i, tweet in enumerate(amostra_elogios, 1):
    # A correção é remover as aspas extras no final da linha abaixo.
    print(f"{i}: {tweet}\n")

### 3. Refinando a Classificação: Focando Apenas nos Clientes

Nossa validação manual revelou um insight importante: o classificador está identificando corretamente as palavras-chave, mas está sendo aplicado a todos os tweets, incluindo os dos agentes de suporte. Isso gera falsos positivos (e.g., um agente que menciona a palavra "issue" tem seu tweet classificado como uma reclamação).

Para criar uma feature mais precisa, vamos refinar nossa abordagem: aplicaremos a lógica de classificação **apenas nos tweets de entrada (inbound)**, que são os que vêm dos clientes.

In [ ]:
# --- Criando uma Feature de Intenção Refinada ---

def classificar_intencao_cliente(row):
    """
    Aplica a lógica de classificação apenas se o tweet for de um cliente.
    Recebe uma linha inteira do DataFrame como entrada.
    """
    # Se o tweet for inbound (do cliente), aplicamos nossa lógica.
    if row['inbound']:
        texto = str(row['text']).lower()
        palavras_reclamacao = ['not working', 'problem', 'issue', 'broken', "doesn't work", 'worst', 'unacceptable', 'fail']
        palavras_elogio = ['thanks', 'great', 'awesome', 'love', 'excellent', 'perfect', 'thank you', 'kudos']
        
        if any(p in texto for p in palavras_reclamacao):
            return 'reclamacao_cliente'
        elif '?' in texto:
            return 'duvida_cliente'
        elif any(p in texto for p in palavras_elogio):
            return 'elogio_cliente'
        else:
            return 'outro_cliente'
    # Se o tweet não for inbound (da empresa), simplesmente rotulamos como 'resposta_empresa'.
    else:
        return 'resposta_empresa'

# Aplicamos a nova função ao DataFrame inteiro, linha por linha (axis=1)
df_filtered['tipo_refinado'] = df_filtered.apply(classificar_intencao_cliente, axis=1)

# Verificamos a nova distribuição
print("--- Distribuição das Intenções Refinadas ---")
print(df_filtered['tipo_refinado'].value_counts())

# Parte 4: Engenharia de Features II - Análise de Sentimento

A segunda feature crucial que vamos criar é o **sentimento** de cada tweet. Saber se um cliente está feliz, neutro ou insatisfeito é um contexto poderoso para gerar respostas automáticas adequadas.

| Ferramenta     | Justificativa Técnica                                                                                               |
| :------------- | :------------------------------------------------------------------------------------------------------------------ |
| **VADER (NLTK)** | Escolhido por ser um modelo de análise de sentimento otimizado para textos curtos de redes sociais, **capaz de interpretar emojis, pontuação e a intensidade das palavras (e.g., "very good!!!")**. |
| **Visualizações**| Serão usadas para analisar a distribuição geral dos sentimentos e, mais importante, a **correlação entre a intenção do cliente e o seu sentimento**. |
| **Integração Futura**| A feature `sentimento` será um input direto para o nosso modelo de LLM, permitindo a geração de respostas contextuais. |

In [ ]:
# --- 1. Implementação da Análise de Sentimentos ---

# Importar as bibliotecas e inicializar as ferramentas
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
analisador = SentimentIntensityAnalyzer()

# Definir a função para rotular o sentimento
def rotular_sentimento(texto):
    """Analisa um texto e o classifica como 'positivo', 'negativo' ou 'neutro'."""
    scores = analisador.polarity_scores(str(texto))
    compound_score = scores['compound']
    if compound_score >= 0.05: return 'positivo'
    elif compound_score <= -0.05: return 'negativo'
    else: return 'neutro'

# Aplicar a função ao DataFrame filtrado, criando a coluna 'sentimento'
# <<< CORREÇÃO DE CONSISTÊNCIA: Usamos 'df_filtered'.
# Aplicamos no texto original ('text') para que VADER use emojis e pontuação.
df_filtered['sentimento'] = df_filtered['text'].apply(rotular_sentimento)

# --- 2. Visualização da Distribuição Geral ---
plt.figure(figsize=(10, 6))
ax = sns.countplot(x='sentimento', data=df_filtered, order=['positivo', 'neutro', 'negativo'], palette='coolwarm_r')
ax.set_title('Distribuição Geral dos Sentimentos nos Tweets', fontsize=16)
ax.set_xlabel('Sentimento', fontsize=12)
ax.set_ylabel('Quantidade de Tweets', fontsize=12)

# Adicionar rótulos de dados para clareza
for p in ax.patches:
    ax.annotate(f'{int(p.get_height()):,}', (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', xytext=(0, 9), textcoords='offset points')

plt.show()

In [ ]:
# Cruzar tipo de mensagem x sentimento
import pandas as pd

df_clientes = df_filtered[df_filtered['inbound'] == True]
tabela_cruzada = pd.crosstab(df_clientes['tipo_refinado'], df_clientes['sentimento'], normalize='index') * 100
tabela_cruzada = tabela_cruzada.round(1)


# Visualizar com gráfico de barras empilhadas
tabela_cruzada.plot(kind='bar', stacked=True, figsize=(10,6), colormap='coolwarm')
plt.title('Sentimentos por Tipo de Mensagem (%)')
plt.xlabel('Tipo de Mensagem')
plt.ylabel('Porcentagem')
plt.legend(title='Sentimento')
plt.grid(True)
plt.show()


### Insights da Análise de Sentimentos

As visualizações acima nos fornecem uma compreensão profunda do tom emocional das interações no nosso dataset.

#### 1. Distribuição Geral dos Sentimentos

O primeiro gráfico (distribuição geral) revela um panorama interessante:
*   **Predominância Positiva:** A maioria dos tweets no dataset tem um sentimento **positivo**. Isso pode ser um pouco contraintuitivo, já que muitas vezes associamos o suporte ao cliente a problemas.
*   **Volume Relevante de Negatividade:** Apesar da predominância positiva, existe um volume substancial de quase **660 mil tweets negativos**, o que representa uma enorme quantidade de clientes insatisfeitos que precisam de atenção.

Essa visão geral, no entanto, é incompleta. Ela mistura tweets de clientes com respostas de empresas. A verdadeira análise vem do cruzamento com a intenção do cliente.

#### 2. Correlação entre Intenção do Cliente e Sentimento

O segundo gráfico (barras empilhadas) é a análise mais poderosa e valida nossas hipóteses:
*   **Reclamações são Negativas:** Como esperado, a categoria `reclamacao_cliente` é majoritariamente composta por sentimentos **negativos** (cerca de 55%). Isso confirma que nosso classificador de intenção está funcionando bem.
*   **Elogios são Positivos:** De forma esmagadora, a categoria `elogio_cliente` é quase **100% positiva**. Outra forte validação da nossa engenharia de features.
*   **Dúvidas e Outros:** As categorias `duvida_cliente` e `outro_cliente` são mais mistas, com uma distribuição mais equilibrada entre sentimentos positivos, neutros e negativos. Isso faz sentido, pois uma dúvida pode ser formulada de maneira neutra ("como faço X?"), frustrada ("por que não consigo fazer X?!") ou positiva.

**Conclusão Estratégica:**
Conseguimos criar duas features (`tipo_refinado` e `sentimento`) que não apenas estruturam os dados, mas também se validam mutuamente. Agora temos um contexto rico para cada tweet de cliente, o que será fundamental para a próxima e última etapa do projeto: **treinar um modelo de IA para gerar respostas adequadas a cada cenário**.

### Análise Bônus: Ranking de Sentimento por Empresa

Como uma análise de negócio adicional, podemos calcular o sentimento médio associado às **respostas** de cada empresa. Isso nos permite criar um ranking e identificar quais marcas se comunicam de forma mais positiva ou negativa.

In [ ]:
# --- Ranking de Sentimento Médio por Empresa ---

# 1. Criar a coluna com a pontuação 'compound' bruta no nosso DataFrame principal.
df_filtered['compound_score'] = df_filtered['text'].apply(lambda text: analisador.polarity_scores(str(text))['compound'])

# 2. Filtrar apenas as respostas das empresas ('outbound') e calcular a média do score.
sentimento_por_empresa = df_filtered[df_filtered['inbound'] == False].groupby('author_id')['compound_score'].mean().sort_values(ascending=False)

# 3. Visualizar as 10 empresas com as respostas mais positivas.
plt.figure(figsize=(12, 6))
sentimento_por_empresa.head(10).plot(kind='barh', color='green')
plt.title('Top 10 Empresas com Maior Sentimento Médio em suas Respostas', fontsize=16)
plt.xlabel('Sentimento Médio (Compound Score)', fontsize=12)
plt.ylabel('ID da Empresa', fontsize=12)
plt.gca().invert_yaxis()
plt.show()

# 4. Visualizar as 10 empresas com as respostas mais negativas.
plt.figure(figsize=(12, 6))
sentimento_por_empresa.tail(10).plot(kind='barh', color='red')
plt.title('Top 10 Empresas com Menor Sentimento Médio em suas Respostas', fontsize=16)
plt.xlabel('Sentimento Médio (Compound Score)', fontsize=12)
plt.ylabel('ID da Empresa', fontsize=12)
plt.gca().invert_yaxis()
plt.show()

### Mapa de Calor: Correlação entre Intenção e Sentimento

Agora, vamos conectar nossas duas novas features (`tipo_refinado` e `sentimento`). Qual é o sentimento predominante em cada tipo de intenção do cliente? Esta análise é fundamental para validar nossas classificações.

In [ ]:
# --- Mapa de Calor: Correlação entre Intenção e Sentimento ---

# 1. Filtrar apenas os tweets que são de clientes ('inbound').
df_clientes = df_filtered[df_filtered['inbound'] == True].copy()

# 2. Criar uma tabela cruzada (crosstab) usando nossa coluna 'tipo_refinado'.
#    <<< CORREÇÃO PRINCIPAL AQUI >>>
cross_tab = pd.crosstab(index=df_clientes['tipo_refinado'], 
                        columns=df_clientes['sentimento'],
                        normalize="index") * 100

# 3. Gerar o Mapa de Calor (Heatmap) para uma visualização impactante.
plt.figure(figsize=(12, 7))
sns.heatmap(cross_tab, annot=True, cmap='viridis', fmt='.1f',
            cbar_kws={'label': 'Porcentagem (%)'})
plt.title('Mapa de Calor: Sentimento (%) por Tipo de Intenção do Cliente', fontsize=16)
plt.xlabel('Sentimento', fontsize=12)
plt.ylabel('Tipo de Intenção (Cliente)', fontsize=12)
plt.show()

### Análise de Performance por Marca e Validação Cruzada

As análises finais desta seção nos permitem avaliar a performance de comunicação das empresas e validar a coerência das nossas features.

#### 1. Ranking de Sentimento por Empresa

Os gráficos de barras horizontais nos mostram um ranking claro de quais empresas utilizam uma linguagem mais positiva ou negativa em suas respostas de suporte.
*   **Melhores Comunicadores:** Empresas como **HPSupport** e **TwitterSupport** se destacam por usarem uma linguagem consistentemente positiva em suas interações.
*   **Pontos de Melhoria:** Marcas como **KFC_UKI_Help** e **SW_Help** apresentam um sentimento médio negativo em suas respostas, o que pode indicar um tom mais seco, robótico ou até mesmo frustrado, sendo um ponto de atenção para a gestão de marca.

Esta análise poderia, em um cenário de negócio real, ser usada para identificar benchmarks de comunicação ou áreas que necessitam de treinamento.

#### 2. Mapa de Calor: A Prova Final

O mapa de calor é a validação definitiva da nossa engenharia de features. Ele mostra a distribuição percentual de sentimentos dentro de cada tipo de intenção do cliente, revelando padrões claros:
*   **Elogios são Positivos (90.4%):** Como esperado, a vasta maioria dos elogios de clientes tem um sentimento detectado como positivo.
*   **Reclamações são Negativas (55.4%):** A categoria `reclamacao_cliente` é predominantemente negativa. Isso não só valida nosso classificador de intenção, mas também mostra que a análise de sentimentos está capturando a frustração dos clientes. O fato de não ser 100% negativo é interessante, indicando que algumas reclamações são feitas de forma mais neutra ou até educada ("... aprecio a ajuda, mas ainda não funcionou").
*   **Dúvidas e Outros:** As outras categorias são mais distribuídas, o que é esperado, pois perguntas e comentários gerais podem ser feitos em qualquer tom.

**Conclusão Final da Engenharia de Features:**
Com sucesso, transformamos textos brutos e não estruturados em **duas features de alta qualidade e mutuamente validadas**: `tipo_refinado` (a intenção do cliente) e `sentimento` (o tom emocional).

Estamos agora na melhor posição possível para avançar para a fase final e mais empolgante do projeto: **utilizar essas features para guiar um modelo de IA Generativa na etapa de Modelagem**.


# Parte 2: Modelagem - Geração de Respostas com LLM

Com nossas features de contexto (`tipo_refinado` e `sentimento`) prontas, entramos na fase de modelagem. O objetivo aqui é construir um sistema que utilize um **Modelo de Linguagem Grande (LLM)** para gerar respostas de atendimento ao cliente que sejam não apenas relevantes, mas também contextualmente adequadas ao tom e à intenção de cada tweet.

### 1. Escolha da Ferramenta e do Modelo

| Tecnologia | Justificativa Técnica                                                                                                                                                                     |
| :----------- | :---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **`transformers` (Hugging Face)** | É a biblioteca padrão da indústria para trabalhar com modelos de linguagem de última geração. Ela nos dá acesso a milhares de modelos pré-treinados e ferramentas para controlá-los. |
| **`gpt2`** | Após experimentação inicial com modelos de diálogo (`DialoGPT`), escolhemos o `gpt2` como nosso modelo final. Por ser um modelo de propósito geral, ele demonstrou maior flexibilidade para seguir os padrões e instruções complexas do nosso prompt, resultando em respostas de maior qualidade para esta tarefa. |
---

### 🧠 Um Mergulho Rápido nos Transformers: O Que Roda Por Baixo dos Panos?

Embora a biblioteca `transformers` facilite nosso trabalho com uma única chamada de `pipeline`, por trás dela ocorre um processo sofisticado, baseado na revolucionária arquitetura "Transformer". Entender seus fundamentos nos permite criar prompts melhores e depurar problemas de forma mais eficaz.

Aqui está uma visão simplificada do que acontece quando pedimos para a nossa IA gerar uma resposta:

1.  **Tokenização:** O texto de entrada (nosso "prompt") não é lido diretamente. Primeiro, ele é quebrado em pedaços menores chamados **tokens**. Estes podem ser palavras inteiras (como "app"), partes de palavras (como "un-accept-able") ou pontuação.

2.  **Embeddings (Conversão para Vetores):** Cada token é então mapeado para um **vetor numérico** de alta dimensão. Este vetor, chamado de *embedding*, representa o "significado" do token em um espaço matemático. Palavras com significados semelhantes terão vetores próximos.

3.  **O Mecanismo de Auto-Atenção (Self-Attention):** Este é o ingrediente secreto e a grande inovação da arquitetura Transformer. Antes de gerar uma resposta, o modelo precisa entender o **contexto**. O mecanismo de atenção permite que o modelo pese a importância de cada token na frase em relação a todos os outros.
    *   **Analogia:** Imagine que você está lendo a frase: *"O aplicativo parou de funcionar depois da atualização, que decepção!"*. Para entender a frustração, o modelo precisa dar mais "atenção" às palavras **'parou'**, **'funcionar'**, **'atualização'** e **'decepção'** do que a 'O' ou 'de'. O mecanismo de atenção essencialmente "ilumina" as palavras mais relevantes para o contexto, permitindo que o modelo "preste atenção" ao que realmente importa.

4.  **Geração da Resposta (Processo Autoregressivo):** Com o contexto entendido, o modelo começa a gerar a resposta, um token de cada vez. Ele prevê a palavra mais provável para vir a seguir. Essa nova palavra é então adicionada à sequência, e o processo se repete, com o modelo prestando atenção em toda a sequência (prompt original + palavras já geradas) para prever a próxima palavra, e assim por diante.

5.  **Decodificação:** Finalmente, a sequência de tokens gerada pela IA é decodificada de volta para um **texto legível**, que é o que vemos como a resposta final.

Entender esse fluxo nos ajuda a compreender por que o "prompt engineering" e técnicas como o "Few-Shot Prompting" são tão eficazes: estamos, na verdade, fornecendo um contexto inicial muito mais rico para o mecanismo de atenção operar.

In [ ]:
# --- Carregando o Modelo Final: GPT-2 ---

# Importar as bibliotecas necessárias
from transformers import pipeline
import torch

# Carregar o pipeline com o modelo 'gpt2', que é mais robusto para seguir instruções.
print("Carregando o modelo de IA 'gpt2'...")
generator = pipeline('text-generation', model='gpt2', device=0 if torch.cuda.is_available() else -1)
print("\nModelo de IA carregado com sucesso!")

### 2. Construindo o Cérebro da Solução: A Função de Geração Contextual

Esta função representa o coração do sistema. Em vez de simplesmente passar o tweet do cliente para o modelo, foi construído um **prompt dinâmico** para instruí-lo sobre como se comportar.

Após um processo de experimentação, concluiu-se que a técnica mais eficaz para o modelo `gpt2` nesta tarefa é o **"One-Shot Prompting"**. A abordagem consiste em fornecer ao modelo um único exemplo de alta qualidade de uma interação ideal (Cliente -> Agente).

Isso "mostra" ao modelo o tom, o formato e a qualidade da resposta esperada, forçando-o a seguir um padrão conversacional e, consequentemente, a gerar resultados muito mais consistentes do que apenas através de instruções diretas.

In [ ]:
# --- Função de Geração Final (com Prompt Híbrido "One-Shot") ---

def gerar_resposta_final(tweet_texto):
    """
    Gera uma resposta usando um prompt híbrido que dá um exemplo claro (One-Shot),
    uma técnica muito eficaz para modelos como o GPT-2.
    """
    # Este prompt define o contexto e mostra um exemplo ideal de interação.
    prompt = f"""This is a transcript of a customer service chat.

###
Customer: My order hasn't arrived yet and the tracking number doesn't work!
Agent: I'm so sorry to hear about the trouble with your order. I can definitely look into this for you. Could you please provide your order number?
###
Customer: {tweet_texto}
Agent: We're sorry"""
    
    respostas = generator(
        prompt,
        max_new_tokens=60,      # Limita o tamanho da resposta
        no_repeat_ngram_size=2, # Evita repetições
        pad_token_id=generator.tokenizer.eos_token_id,
        truncation=True
    )

    # Lógica de limpeza para extrair apenas a resposta final do agente.
    texto_gerado = respostas[0]['generated_text']
    # Encontra a posição da última ocorrência de "Agent:"
    posicao_final_agent = texto_gerado.rfind("Agent:")
    # Pega todo o texto após essa posição.
    resposta_limpa = texto_gerado[posicao_final_agent + len("Agent:"):].strip()
    
    return resposta_limpa



### 3. Teste em Cenários Reais

Para validar o sistema completo, a etapa seguinte consiste em testá-lo com dados reais. Serão selecionadas amostras aleatórias de diferentes intenções de cliente (reclamações, elogios, etc.) do dataset para serem processadas pela função de geração de IA, permitindo uma avaliação qualitativa das respostas geradas.

In [ ]:
# --- Teste Final com a Abordagem Definitiva ---

# --- Teste 1: Uma Reclamação REAL de um cliente ---
exemplo_reclamacao = df_filtered[df_filtered['tipo_refinado'] == 'reclamacao_cliente'].sample(1)
texto_reclamacao = exemplo_reclamacao['text'].values[0]
resposta_gerada_reclamacao = gerar_resposta_final(texto_reclamacao)

print("--- 💬 EXEMPLO COM RECLAMAÇÃO ---")
print(f"Tweet Original: {texto_reclamacao}")
print(f"✅ Resposta Gerada pelo LLM: {resposta_gerada_reclamacao}\n")


# --- Teste 2: Um Elogio REAL de um cliente ---
exemplo_elogio = df_filtered[df_filtered['tipo_refinado'] == 'elogio_cliente'].sample(1)
texto_elogio = exemplo_elogio['text'].values[0]
resposta_gerada_elogio = gerar_resposta_final(texto_elogio)

print("--- 😍 EXEMPLO COM ELOGIO ---")
print(f"Tweet Original: {texto_elogio}")
print(f"✅ Resposta Gerada pelo LLM: {resposta_gerada_elogio}\n")


# --- Teste 3: Uma Dúvida REAL de um cliente ---
exemplo_duvida = df_filtered[df_filtered['tipo_refinado'] == 'duvida_cliente'].sample(1)
texto_duvida = exemplo_duvida['text'].values[0]
resposta_gerada_duvida = gerar_resposta_final(texto_duvida)

print("--- ❓ EXEMPLO COM DÚVIDA ---")
print(f"Tweet Original: {texto_duvida}")
print(f"✅ Resposta Gerada pelo LLM: {resposta_gerada_duvida}\n")

# Conclusão e Próximos Passos

## ✅ Resultados e Conclusão do Projeto

Neste projeto, foi desenvolvido com sucesso um protótipo de ponta a ponta para um sistema de otimização de atendimento ao cliente baseado em IA. Partindo de um dataset bruto de quase 3 milhões de tweets, foi possível:

1.  **Analisar e Pré-processar** os dados, identificando o período de maior relevância (2017) e aplicando técnicas de limpeza de texto.
2.  **Criar Features de Alto Valor** através da Engenharia de Features, classificando a intenção de cada tweet de cliente (`tipo_refinado`) e seu tom emocional (`sentimento`).
3.  **Implementar um Modelo de Linguagem Avançado (LLM)**, o **`gpt2`**, para gerar respostas contextuais.
4.  **Refinar a Geração de Respostas** utilizando a técnica de **"One-Shot Prompting"** e o controle de parâmetros (`temperature`, `top_k`, etc.), melhorando significativamente a qualidade e a consistência das respostas geradas.

O resultado final é um sistema capaz de receber um tweet e, em segundos, fornecer uma sugestão de resposta adequada, demonstrando o imenso potencial da IA Generativa para automatizar e escalar operações de atendimento ao cliente.

## 🚀 Próximos Passos e Melhorias Futuras

Este protótipo funcional é um excelente ponto de partida, mas um projeto de IA está sempre em evolução. Com base no que foi construído, identifico várias oportunidades claras para levar esta solução ao próximo nível:

*   **Evoluir a Classificação de Intenção:** O classificador por regras foi um ótimo primeiro passo, mas para capturar nuances mais complexas, o próximo passo seria **treinar um modelo de Machine Learning dedicado**. Eu poderia começar com um `Logistic Regression` sobre vetores TF-IDF e, para máxima precisão, evoluir para um modelo baseado em Transformers, como o `BERT`, que treinaria especificamente para esta tarefa.

*   **Personalizar o Tom com Fine-tuning:** Para que as respostas soem exatamente como uma marca específica, eu realizaria o **fine-tuning (ajuste fino) do modelo `gpt2`**. Para isso, eu buscaria um dataset com exemplos reais de interações da empresa, ensinando o modelo a adotar um tom de voz e um estilo de comunicação únicos.

*   **Aumentar a Qualidade com Modelos de Ponta:** Para alcançar o estado da arte em geração de texto, o passo seguinte seria integrar o sistema com APIs de modelos maiores, como o **GPT-4 (OpenAI)** ou o **Gemini (Google AI)**. Isso me permitiria explorar como a qualidade da resposta escala com o poder do modelo e me daria experiência prática com plataformas de MLaaS (Machine Learning as a Service) como **AWS Bedrock** ou **Azure AI Studio**.

*   **Transformar em Produto com um Dashboard Interativo:** A melhor forma de demonstrar o valor deste projeto é torná-lo tangível. Meu próximo objetivo é **empacotar esta solução em um dashboard interativo usando Streamlit**. Isso permitiria que qualquer pessoa, mesmo sem conhecimento técnico, pudesse testar a IA, ver os resultados e entender seu potencial.

*   **Quantificar o Impacto de Negócio:** Finalmente, em um contexto corporativo, eu focaria em uma **análise de custo-benefício**. Isso envolveria estimar o impacto da automação em métricas de negócio, como a potencial redução no Tempo Médio de Resposta (TMR) e o cálculo do Retorno Sobre o Investimento (ROI) da solução.